In [8]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np

<h1>Part 1</h1>

<b>Data Scraping</b>

In [9]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'lxml')

# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [10]:
# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [11]:
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"Postal Code": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,Postal Code,Borough,Neighborhood
0,\nM1ANot assigned,\nM2ANot assigned,\nM3ANorth York(Parkwoods)
1,\nM1BScarborough(Malvern / Rouge),\nM2BNot assigned,\nM3BNorth York(Don Mills)North
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned,\nM3CNorth York(Don Mills)South(Flemingdon Park)
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned,\nM3ENot assigned
4,\nM1GScarborough(Woburn),\nM2GNot assigned,\nM3GNot assigned


In [12]:
#drop cells with Borough not assigned
toronto_dropped = toronto_df[toronto_df.Borough != 'Not assigned'].reset_index(drop=True)
toronto_dropped.head()

,Postal Code,Borough,Neighborhood
0,\nM1ANot assigned,\nM2ANot assigned,\nM3ANorth York(Parkwoods)
1,\nM1BScarborough(Malvern / Rouge),\nM2BNot assigned,\nM3BNorth York(Don Mills)North
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned,\nM3CNorth York(Don Mills)South(Flemingdon Park)
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned,\nM3ENot assigned
4,\nM1GScarborough(Woburn),\nM2GNot assigned,\nM3GNot assigned


In [13]:
# group multiple neighborhoods having same postal code
toronto_group = toronto_dropped.groupby(['Postal Code', 'Borough'], as_index=False).agg(lambda x: ", ".join(x))
toronto_group['Neighborhood'] = toronto_group['Neighborhood'].str.replace('/', ',')
toronto_group.head()

,Postal Code,Borough,Neighborhood
0,\nM1ANot assigned,\nM2ANot assigned,\nM3ANorth York(Parkwoods)
1,\nM1BScarborough(Malvern / Rouge),\nM2BNot assigned,\nM3BNorth York(Don Mills)North
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned,\nM3CNorth York(Don Mills)South(Flemingdon Park)
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned,\nM3ENot assigned
4,\nM1GScarborough(Woburn),\nM2GNot assigned,\nM3GNot assigned


In [ ]:
# rows of dataframe
toronto_group.shape